In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit import QuantumCircuit
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier

from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN

from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime import Options

from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_provider import least_busy
from qiskit.utils import QuantumInstance

from qiskit_ibm_runtime import QiskitRuntimeService

# Set up the provider and device
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q-asu/main/pi-deluca',
)
backend = service.backend('simulator_statevector')
sampler = Sampler(session=backend)

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(X)

num_classes = len(np.unique(y))
print("num_classes:", num_classes)

num_qubits = X.shape[1]
print("num_qubits:", num_qubits)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

# Define the quantum feature map
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2)

# construct ansatz
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=2)

# construct quantum circuit
qc = QuantumCircuit(num_qubits)
qc.append(feature_map, range(num_qubits))
qc.append(ansatz, range(num_qubits))
qc.decompose().draw(output="mpl")

def parity(X):
    return "{:b}".format(X).count("1") % num_classes

# Define the classical machine learning model
sampler = Sampler()

svm = SamplerQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=parity,
    output_shape=num_classes,
    sampler=sampler
)

# construct classifier
sampler_classifier = NeuralNetworkClassifier(
    neural_network=svm, optimizer=COBYLA(maxiter=30), one_hot=True,
)

# Fit the model on the training data
sampler_classifier.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = sampler_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


num_classes: 3
num_qubits: 4


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Accuracy: 0.6981132075471698
